# Manipulation des variables de décision.

Dans ce notebook, on va voir comment créer des variables de décision rapidement dans un solveur python.

On va aussi voir comment itérer.

On va aussi voir comment utiliser un dictionnaire de variables de décision quand on 
a énormément de données à optimiser.

## Sommaire

1. <b>Méthode de base</b>
2. <b>Méthode avec le dictionnaire Python Pulp</b>
3. <b>Créer des contraintes plus rapidement avec les dictionnaires pulp.</b>
4. <b>Utiliser un dictionnaire de variables de décisions dans la fonction objectif</b>
    - Le problème à traiter : Minimisation de coût en livraison (Transport).
    - Solutionner en écrivant à la main les variables de décision 
        - Contraintes lignes
        - Curiosité : On change les contraintes
        - Contraintes colonnes
    - Solutionner avec les itérations et dictionnaires de vars de décision
    - Importer un fichier Excel
5. <b>Utiliser une matrice dans les contraintes</b>
6. <b>Itérer</b>
    - avec le solveur Python Gekko 
    - avec le solveur Python Cplex
    - avec le solveur Python Cvxopt

Etude globale proposée par <b>Estelle Derrien - Github estellederrien</b>

# 1. Méthode de base

Rappel :
Une variable de décision contient une valeur qui sera ensuite calculée par le solveur pour être la meilleure valeur possible pour minimiser ou maximiser une fonction objectif ( C'est ça, l'optimisation)

In [1]:
# La méthode de base :

# -----------------------------------
# Import de Python Pulp
# -----------------------------------
from pulp import *

# -----------------------------------
# Type du problème
# -----------------------------------
# On choisit de résoudre un problème de Maximisation
Lp_prob = LpProblem('Problem', LpMaximize)

# -----------------------------------
# Variables de décision
# -----------------------------------
# On définit nos variables de décision ci-dessous

x = LpVariable("x", lowBound=0, cat='Integer')   # Créer une variable x >= 0
y = LpVariable("y", lowBound=0, cat='Integer')   # Créer une variable y >= 0

# -----------------------------------
# Affichage
# -----------------------------------
print(x)
print(y)

x
y


# 2. Méthode avec le dictionnaire de Python Pulp

Dans l'exemple précédent, ca va parce qu'on a que deux variables de décisions, mais qu'en sera-t-il si on en a 100 ?
On peut aussi utiliser cette méthode pour créer plus rapidement des variables de décision.


In [2]:
# -----------------------------------
# Import de Python Pulp
# -----------------------------------
from pulp import *

# -----------------------------------
# Type du problème
# -----------------------------------
prob = LpProblem ("MaximiserProfit", LpMaximize)

# -----------------------------------
# Nos données
# -----------------------------------
produits = ["automobile", "cycle1","cycle2","dragon","nounours","poupee","arc"]

# -----------------------------------
# Variables de décision
# -----------------------------------
# Ici, on crée nos variable de décision en créant un "dicts".
x = LpVariable.dicts("produits ", produits , 0)

# -----------------------------------
# Impression
# -----------------------------------
# On visualise notre dicts
# Chaque produit est désormais une variable de décision qui peut prendre une valeur.
print(x)


{'automobile': produits__automobile, 'cycle1': produits__cycle1, 'cycle2': produits__cycle2, 'dragon': produits__dragon, 'nounours': produits__nounours, 'poupee': produits__poupee, 'arc': produits__arc}


# 3. Créer des contraintes plus rapidement avec les dictionnaires

Dans cet exemple ci-dessous, on crée des valeurs de contraintes dans des dictionnaires python afin de pouvoir <b>itérer</b> dessus ensuite, pour aller plus vite.

En fait, c'est aussi une association de valeurs à nos variables de décisions, réalisées aux format dictionnaire python, ce qui permet d'écrire plus vite ensuite dans la fonction objectif et dans les contraintes. 

Mais attention, le nombre de variables de décisions doit correspondre au nombre dans les dictionnaires Python , ici, on a 7 variables de décisions, donc dans les dictionnaires, on retrouve cette taille de 7 variables de décision.

Ceci peut faciliter les choses dans le cas ou l'on importe ces données à partir d'une base de données, il devient plus facile de les placer dans un dictionnaire.


In [3]:
# -----------------------------------
# Import de Pulp
# -----------------------------------
from pulp import *

# -----------------------------------
# Nos données
# -----------------------------------

# La liste de nos produits ( des jouets), ca va être les variables de décision, 
# elles pourront prendre une valeur entière, vu que ce sont des objets uniques. 
# Par exemple , produire 15 automobiles, produire 25 arcs etc ...
produits = ["automobile", "cycle1","cycle2","dragon","nounours","poupee","arc"]

# Les bénéfices en EUROS par produits
benefices = {"automobile": 8, "cycle1": 12, "cycle2": 14,"dragon": 3,"nounours":6,"poupee":13,"arc":12}

# Emplois (en kgs)
plastique = {"automobile": 2, "cycle1": 4, "cycle2": 5,"dragon": 3,"nounours":1,"poupee":4,"arc":2}
bois      = {"automobile": 1, "cycle1": 1, "cycle2": 2,"dragon": 2,"nounours":1,"poupee":5,"arc":1}
acier     = {"automobile": 1, "cycle1": 2, "cycle2": 3,"dragon": 3,"nounours":2,"poupee":2,"arc":5}

# Les noms de nos ressources
ressources = {"plastique", "bois", "acier"}

# Les stocks de nos ressources en KG
stocks = {"plastique": 142, "bois ": 117, "acier": 124}

# -----------------------------------
# Type du problème
# -----------------------------------

# Créer le type de problème, ici, on veut maximiser notre profit de notre usine de jouets
prob = LpProblem ("MaximiserProfit", LpMaximize)

# -----------------------------------
# Variables de décision
# -----------------------------------

# On crée nos variables, en se basant sur le tableau x ( array en Anglais)
# On stipule que ce sont des variables entières, normal, puisque ce sont des jouets (On ne peut pas avoir 1/2 jouet...).
x = LpVariable.dicts("produits ", produits , lowBound=0, cat='Integer')

# -----------------------------------
# Fonction objectif
# -----------------------------------

# La fonction objectif, Maximiser le bénéfice.
# Ici, on voit qu'on itère à l'aide de notre tableau produits, sur les bénéfices qui sont contenus dans un 
# dictionnaire Python. Cette méthode revient souvent sur StackOverflow et en général.
prob += lpSum([benefices[i] * x[i] for i in produits ]), "MaximiserBenefice" 

# -----------------------------------
# Contraintes
# -----------------------------------

# Nos contraintes.
# On respecte notre production sous contrainte de stocks
# Ici, n voit qu'on itère à l'aide de notre tableau produits, sur les stocks qui sont contenus dans un 
# dictionnaire Python. Cette méthode revient souvent sur StackOverflow et en général.
prob += lpSum([plastique[i] * x[i] for i in  produits]) <= 142 ,"MaxPlastique"

# La ligne suivante veut dire : 
# "" Prends chaque valeur du tableau bois indicé par i et multiplie le par la 
# variable de décision x indice i, fait la somme de tout
# et cette somme doit être inférieure à notre stock de bois de 117
prob += lpSum([bois[i]      * x[i] for i in  produits]) <= 117 ,"MaxBois"

prob += lpSum([acier[i]     * x[i] for i in  produits]) <= 124 ,"MaxAcier"

# Production minimale par produits pour les clients : 2 unités
for p in produits:
   prob += x[p] >= 2, f"min production units for product {p}"


# -----------------------------------
# Solution
# -----------------------------------
   
# On utilise le solver pulp
prob.solve()

# On affiche le statut de la solution
print ("Status:", LpStatus [prob.status])

# On écrit aussi le probleme dans un fichier
# prob.writeLP ( "JouetsModel.lp")

# Afficher l'optimium de chaques variables produits qui s'exprime en unité construites
for v in prob.variables ():
    print (v.name, "=", v.varValue)


# Le résultat de la fonction objectif est ici :
print ("TotalProfit", value (prob.objective))
  

Status: Optimal
produits__arc = 2.0
produits__automobile = 40.0
produits__cycle1 = 2.0
produits__cycle2 = 2.0
produits__dragon = 2.0
produits__nounours = 26.0
produits__poupee = 2.0
TotalProfit 584.0


# 4. Utiliser un dictionnaire de variables de décisions dans la fonction objectif

Pour comprendre cette technique, on va utiliser un problème simplifié de minimisation de coût en livraison, généralement appelé problème de Transport, (Transportation, en Anglais).

- J'ai 4 fournisseurs et 4 dépots.
- Chaque trajet entre un fournisseur et un dépot a un coût exprimé en euros.
- Je veux minimiser le coût global de livraison, tout en m'assurant que <b> chaque fournisseur</b> soit utilisé, je ne suis pas obligé de livrer chaque dépot.

On peut aussi en retrouver un similaire dans mon fichier 13 " Optimisation chimie et pétrole "avec un import excel 

Voici la matrice des coûts de livraison de notre problème :

In [4]:
                #Depot_A #Depot_B #Depot_C #Depot_D
# Fournisseur_A 350       450       300       250
# Fournisseur_B 450       390       310       268
# Fournisseur_C 600       395       420       290 
# Fournisseur_D 385       700       290       320


Dans le tableau précédent, on voit que transporter une livraison du fournisseur A au dépot A coûte 350 euros, 
mais ne coûte que 250 euros jusqu'au dépot D.


Voici un schéma rudimentaire de toutes les relations possibles, on voit déjà que c'est compliqué à tracer, alors imaginer avec 300 fournisseurs et 300 dépots ...

<div style="text-align:center">
<img src="img/schemaintro.jpg">
</div>

Dans ce problème, nous définissons la variable de décision binaire, X(i, j) telle que

     X (i, j) = 1 si le Fournisseur 'i' est attribué au dépot 'j' sinon 0 pour tous i∈ (1,..4) et j ∈ (1,…4)

In [5]:
# -----------------------------------
# Import de Pulp
# -----------------------------------
from pulp import *

# -----------------------------------
# Nos données
# -----------------------------------

# On commence par créer nos données

fournisseurs = ['Fournisseur_A','Fournisseur_B','Fournisseur_C','Fournisseur_D']

depots = ['Depot_A','Depot_B','Depot_C','Depot_D']

couts = [  

# dépots   # A    B    C    D  
            [350, 450, 300, 250],       # A   fournisseurs
            [450, 390, 250, 268],       # B
            [600, 395, 420, 290],       # C
            [385, 700, 290, 320],       # D
]




## Etape 1 : Résoudre en tapant les variables de décision à la main.

Dans un premier temps, afin de bien comprendre l'intérêt du dictionnaire de variables de décisions, on va résoudre en tapant les variables de décision à la main.

Le but est de bien comprendre combien il est rébarbatif de taper les variables de décision à la main !

AU niveau des contraintes, on a des "contraintes lignes".


In [6]:

# -----------------------------------
# Type du problème
# -----------------------------------

# On déclare un problème de Minimisation avec Python Pulp
model = LpProblem("minimiser_couts", LpMinimize)

# -----------------------------------
# Nos Variables de décision
# -----------------------------------

# On déclare les variables de décision à la main, une par une, c'est vraiment rébarbatif.
# On le fait quand même afin de bien comprendre l'intérêt du dictionnaire de variables de décisions ensuite.
# Ce sont donc chaque association possible entre les dépots et fournisseurs
# FADA veut dire ' Du fournisseur A au dépot A'

FADA = LpVariable("FADA", lowBound=0, cat='Binary')   # Créer une variable binaire
FADB = LpVariable("FADB", lowBound=0, cat='Binary')   # Créer une variable binaire
FADC = LpVariable("FADC", lowBound=0, cat='Binary')   # Créer une variable binaire
FADD = LpVariable("FADD", lowBound=0, cat='Binary')   # Créer une variable binaire

FBDA = LpVariable("FBDA", lowBound=0, cat='Binary')   # Créer une variable binaire
FBDB = LpVariable("FBDB", lowBound=0, cat='Binary')   # Créer une variable binaire
FBDC = LpVariable("FBDC", lowBound=0, cat='Binary')   # Créer une variable binaire
FBDD = LpVariable("FBDD", lowBound=0, cat='Binary')   # Créer une variable binaire

FCDA = LpVariable("FCDA", lowBound=0, cat='Binary')   # Créer une variable binaire
FCDB = LpVariable("FCDB", lowBound=0, cat='Binary')   # Créer une variable binaire
FCDC = LpVariable("FCDC", lowBound=0, cat='Binary')   # Créer une variable binaire
FCDD = LpVariable("FCDD", lowBound=0, cat='Binary')   # Créer une variable binaire

FDDA = LpVariable("FDDA", lowBound=0, cat='Binary')   # Créer une variable binaire
FDDB = LpVariable("FDDB", lowBound=0, cat='Binary')   # Créer une variable binaire
FDDC = LpVariable("FDDC", lowBound=0, cat='Binary')   # Créer une variable binaire
FDDD = LpVariable("FDDD", lowBound=0, cat='Binary')   # Créer une variable binaire

# -----------------------------------
# Fonction objectif
# -----------------------------------

# Maintenant on va créer la fonction objectif coût à minimiser, forcément, 
# sans dictionnaire de décision, ni boucle d'itération,
# C'est très lourd à écrire.

# L'objectif est de réduire la somme du coût global de chaque relation
model += (350 * FADA +  450 * FADB + 300 * FADC + 250 * FADD 
          + 450 * FBDA + 390 * FBDB + 250 * FBDC + 268 * FBDD + 
          600 *  FCDA + 395 * FCDB + 420 * FCDC + 290 * FCDD + 
          385 * FDDA + 700 * FDDB + 290 * FDDC + 320 * FDDD)

# -----------------------------------
# Contraintes
# -----------------------------------

# Les contraintes sont que chaque fournisseur livre au moins un dépot
# Souvent, en Anglais, ils disent que ce sont des "contraintes lignes"
# cette ligne veut dire : 
# Il doit forcément exister une relation entre Le fournisseur A et un des 4 dépots.
# Comme il va minimiser le tout, il va choisir celle ayant la valeur minimum
model += FADA +  FADB + FADC + FADD == 1
# cette ligne de contrainte veut dire : 
# Il doit forcément exister une relation entre Le fournisseur B et un des 4 dépots.
# Comme il va minimiser le tout, il va choisir celle ayant la valeur minimum
model += FBDA +  FBDB + FBDC + FBDD == 1
# cette ligne de contrainte veut dire : 
# Il doit forcément exister une relation entre Le fournisseur C et un des 4 dépots.
# Comme il va minimiser le tout, il va choisir celle ayant la valeur minimum
model += FCDA +  FCDB + FCDC + FCDD == 1
# cette ligne de contrainte veut dire : 
# Il doit forcément exister une relation entre Le fournisseur D et un des 4 dépots.
# Comme il va minimiser le tout, il va choisir celle ayant la valeur minimum
model += FDDA +  FDDB + FDDC + FDDD == 1


# -----------------------------------
# Solution
# -----------------------------------

# On résouds avec Python Pulp, il va minimiser le coût global.
model.solve()

# On affiche la valeur de chaque variable > 0, c'est à dire, la relation selectionnée
for i in model.variables():
    if i.varValue > 0:
        print('Selection :',i.name, '=', i.varValue)

# On affiche la valeur de la fonction objectif coût minimisée
print("Coût total minimisé : Euros {}".format(value(model.objective)))

Selection : FADD = 1.0
Selection : FBDC = 1.0
Selection : FCDD = 1.0
Selection : FDDC = 1.0
Coût total minimisé : Euros 1080.0


Donc, le résultat est que le solveur selectionne bien les valeurs minimales pour chaque  relation Fournisseur/Dépot.
Du fournisseur A au dépot D, on a un coût de 250 euros, donc il a choisi cette relation.. etc
Il a donc minimisé le coût total en parcourant les valeurs de la matrice.
On voit qu'il livre plusieurs fois le dépot D, parce qu'on a pas créé de contrainte stipulant qu'on veut livrer chaque dépot une seule fois, et au moins une fois.

## Curiosité 1 : On change nos contraintes.

On va changer les contraintes par curiosité.
Finalement, on ne veut pas forcement que chaque fournisseur soit utilisé , on peut en utiliser un seul, et je ne suis pas obligé d'utiliser chaque dépot mais je veux exactement 4 trajets(routes), tant que cela minimise le coût global, 
comment faire?



In [7]:

# -----------------------------------
# Type du problème
# -----------------------------------
model = LpProblem("minimiser_couts", LpMinimize)

# -----------------------------------
# Variables de décision
# -----------------------------------

FADA = LpVariable("FADA", lowBound=0, cat='Binary')   # Créer une variable binaire
FADB = LpVariable("FADB", lowBound=0, cat='Binary')   # Créer une variable binaire
FADC = LpVariable("FADC", lowBound=0, cat='Binary')   # Créer une variable binaire
FADD = LpVariable("FADD", lowBound=0, cat='Binary')   # Créer une variable binaire

FBDA = LpVariable("FBDA", lowBound=0, cat='Binary')   # Créer une variable binaire
FBDB = LpVariable("FBDB", lowBound=0, cat='Binary')   # Créer une variable binaire
FBDC = LpVariable("FBDC", lowBound=0, cat='Binary')   # Créer une variable binaire
FBDD = LpVariable("FBDD", lowBound=0, cat='Binary')   # Créer une variable binaire

FCDA = LpVariable("FCDA", lowBound=0, cat='Binary')   # Créer une variable binaire
FCDB = LpVariable("FCDB", lowBound=0, cat='Binary')   # Créer une variable binaire
FCDC = LpVariable("FCDC", lowBound=0, cat='Binary')   # Créer une variable binaire
FCDD = LpVariable("FCDD", lowBound=0, cat='Binary')   # Créer une variable binaire

FDDA = LpVariable("FDDA", lowBound=0, cat='Binary')   # Créer une variable binaire
FDDB = LpVariable("FDDB", lowBound=0, cat='Binary')   # Créer une variable binaire
FDDC = LpVariable("FDDC", lowBound=0, cat='Binary')   # Créer une variable binaire
FDDD = LpVariable("FDDD", lowBound=0, cat='Binary')   # Créer une variable binaire

# -----------------------------------
# Fonction objectif
# -----------------------------------

# L'objectif est de réduire la somme du coût global de chaque relation
model += (350 * FADA +  450 * FADB + 300 * FADC + 250 * FADD 
          + 450 * FBDA + 390 * FBDB + 250 * FBDC + 268 * FBDD 
          + 600 *  FCDA + 395 * FCDB + 420 * FCDC + 290 * FCDD 
          + 385 * FDDA + 700 * FDDB + 290 * FDDC + 320 * FDDD)

# -----------------------------------
# Contraintes
# -----------------------------------

# on peut désormais utiliser qu'un seul fournisseur si il faut.
# On a pas du tout à livrer chaque dépot.
# On veut donc que Pulp choisisse exactement 4 routes, et que cela minimise l'intégralité du total des données.
model +=    (FADA +  FADB + FADC  + FADD + 
             FBDA + FBDB + FBDC + FBDD + 
             FCDA + FCDB + FCDC + FCDD + 
             FDDA + FDDB + FDDC + FDDD) == 4

# -----------------------------------
# Solution
# -----------------------------------

# On résouds avec Python Pulp, il va minimiser le coût global.
model.solve()

# On affiche la valeur de chaque variable > 0, c'est à dire, la relation selectionnée
for i in model.variables():
    if i.varValue > 0:
        print('Selection :',i.name, '=', i.varValue)

# On affiche la valeur de la fonction objectif coût minimisée
print("Coût total minimisé : Euros {}".format(value(model.objective)))

Selection : FADD = 1.0
Selection : FBDC = 1.0
Selection : FBDD = 1.0
Selection : FCDD = 1.0
Coût total minimisé : Euros 1058.0


## Curiosité 2 : Contraintes colonne

On va de nouveau changer nos contraintes, pour s'exercer, désormais, <b>chaque Dépôt doit impérativement être livré</b> mais,
le même fournisseur peut être utilisé plusieurs fois!

On appelle ça des contraintes 'colonne', en anglais, parce que quand on lit la matrice, on les déduit en lisant les colonnes.

In [8]:

# -----------------------------------
# Type du problème
# -----------------------------------
model = LpProblem("minimiser_couts", LpMinimize)

# -----------------------------------
# Vars de décision
# -----------------------------------

FADA = LpVariable("FADA", lowBound=0, cat='Binary')   # Créer une variable binaire
FADB = LpVariable("FADB", lowBound=0, cat='Binary')   # Créer une variable binaire
FADC = LpVariable("FADC", lowBound=0, cat='Binary')   # Créer une variable binaire
FADD = LpVariable("FADD", lowBound=0, cat='Binary')   # Créer une variable binaire

FBDA = LpVariable("FBDA", lowBound=0, cat='Binary')   # Créer une variable binaire
FBDB = LpVariable("FBDB", lowBound=0, cat='Binary')   # Créer une variable binaire
FBDC = LpVariable("FBDC", lowBound=0, cat='Binary')   # Créer une variable binaire
FBDD = LpVariable("FBDD", lowBound=0, cat='Binary')   # Créer une variable binaire

FCDA = LpVariable("FCDA", lowBound=0, cat='Binary')   # Créer une variable binaire
FCDB = LpVariable("FCDB", lowBound=0, cat='Binary')   # Créer une variable binaire
FCDC = LpVariable("FCDC", lowBound=0, cat='Binary')   # Créer une variable binaire
FCDD = LpVariable("FCDD", lowBound=0, cat='Binary')   # Créer une variable binaire

FDDA = LpVariable("FDDA", lowBound=0, cat='Binary')   # Créer une variable binaire
FDDB = LpVariable("FDDB", lowBound=0, cat='Binary')   # Créer une variable binaire
FDDC = LpVariable("FDDC", lowBound=0, cat='Binary')   # Créer une variable binaire
FDDD = LpVariable("FDDD", lowBound=0, cat='Binary')   # Créer une variable binaire

# -----------------------------------
# Objectif
# -----------------------------------

# L'objectif est de réduire la somme du coût global de chaque relation
model += (350 * FADA +  450 * FADB + 300 * FADC + 250 * FADD 
          + 450 * FBDA + 390 * FBDB + 250 * FBDC + 268 * FBDD + 
          600 *  FCDA + 395 * FCDB + 420 * FCDC + 290 * FCDD + 
          385 * FDDA + 700 * FDDB + 290 * FDDC + 320 * FDDD)

# -----------------------------------
# Contraintes
# -----------------------------------

# Les contraintes sont que chaque dépot soies impérativement livré.
# On voit que ce sont des contraintes de type "colonne"

model += FADA + FBDA + FCDA + FDDA == 1
model += FADB + FBDB + FCDB + FDDB == 1
model += FADC + FBDC + FCDC + FDDC == 1
model += FADD + FBDD + FCDD + FDDD == 1

# -----------------------------------
# Solution
# -----------------------------------

# On résouds avec Python Pulp, il va minimiser le coût global.
model.solve()

# On affiche la valeur de chaque variable > 0, c'est à dire, la relation selectionnée
for i in model.variables():
    if i.varValue > 0:
        print('Selection :',i.name, '=', i.varValue)

# On affiche la valeur de la fonction objectif coût minimisée
print("Coût total minimisé : Euros {}".format(value(model.objective)))

Selection : FADA = 1.0
Selection : FADD = 1.0
Selection : FBDB = 1.0
Selection : FBDC = 1.0
Coût total minimisé : Euros 1240.0


On voit donc qu'il selectionne FADA, soit la relation fournisseur A à Dépot A, parce qu'il a lu les données en colonne et qu'il a trouvé que la valeur 350 et celle qui minimise la première colonne. Et ainsi de suite pour les 4 colonnes.

## Etape 2 : Résoudre en utilisant le dictionnaire de variables de décision.

On a donc réussi précédemment à obtenir un bon résultat.
Admettons maintenant qu'on nous fournisse un fichier avec 500 fournisseurs et 500 Dépots, va-t-il falloir tout écrire à la main ?
Non, bien sur, on va donc voir les techniques de création de dictionnaires de variables de décision et les méthodes d'itération.

On Crée un dictionnaire des variables de décision automatiquement et on s'en sert dans la fonction objectif, afin de ne pas avoir à taper le code imbuvable de la partie précédente et que tout soit fait automatiquement.

A venir , création en cours

In [9]:

# -----------------------------------
# Variables de décision
# -----------------------------------

# On crée les variables de décisions
# C'est chaque possibilités de relation entre les fournisseurs et les dépots
# On compte s'en servir ensuite parce que si une relation est choisie, la variable
# de décision prendra la valeur de 1, mais si elle ne l'est pas, elle sera à 0
# Elle sera choisie si elle minimise le résultat final.
var_dict = LpVariable.dicts(
    name="relation",
    indices=[(i, j) for i in fournisseurs for j in depots], 
    lowBound=0,
    cat="Binary")

# On affiche les 10 premières pour vérifier que c'est bon.
[var_dict[(i,j)] for i in fournisseurs for j in depots][:10]

[relation_('Fournisseur_A',_'Depot_A'),
 relation_('Fournisseur_A',_'Depot_B'),
 relation_('Fournisseur_A',_'Depot_C'),
 relation_('Fournisseur_A',_'Depot_D'),
 relation_('Fournisseur_B',_'Depot_A'),
 relation_('Fournisseur_B',_'Depot_B'),
 relation_('Fournisseur_B',_'Depot_C'),
 relation_('Fournisseur_B',_'Depot_D'),
 relation_('Fournisseur_C',_'Depot_A'),
 relation_('Fournisseur_C',_'Depot_B')]

# 5. Exemple 2.


Dans cet exemple, on voit que l'auteur de Python Pulp crée les variables de décision en associant 2 tableaux : les fournisseurs et les clients.

Ca permet de créer toutes les solutions possibles à l'aide du dictionnaire de variables de décision.

On s'en sert, après , pendant l'optimisation

In [10]:
import pulp as p

# Variables de décision
fournisseurs = ['A','B']
clients = ['1','2','3','4','5']
costs = [  
# clients   # 1  2  3  4  5
                [3, 1, 3, 2,9],      # A   fournisseurs
                [25, 15, 32, 22,8],  # B
]

# C'est un problème de minimisation
Problem = p.LpProblem('optimisation_transport',p.LpMinimize)

# Créer une liste de toutes les routes possibles
Routes = [(w, b) for w in fournisseurs  for b in clients]

# Créer un dictionnaire de variables de décisions des routes
vars = p.LpVariable.dicts("Route", (fournisseurs , clients), 0, None, p.LpBinary)

# Créer un dictionnaire des coûts de chaque routes
cost = p.makeDict([fournisseurs,clients], costs, 0)

# Fonction objectif
Problem += p.lpSum([cost[w][b]  * vars[w][b]  for (w, b) in Routes])

# Contrainte
Problem += (p.lpSum([ vars[w][b]  for (w, b) in Routes]) >= 5 , "minimum_de_5_routes")

# On résouds
Problem.solve()
 
# On imprime le résultat
print('Statut:', p.LpStatus[Problem.status])
print('Cout total minimisé = ', p.value(Problem.objective))
 
for i in Problem.variables():
    if i.varValue > 0:
        print('Choix de la route :',i.name, '=', i.varValue)


Statut: Optimal
Cout total minimisé =  17.0
Choix de la route : Route_A_1 = 1.0
Choix de la route : Route_A_2 = 1.0
Choix de la route : Route_A_3 = 1.0
Choix de la route : Route_A_4 = 1.0
Choix de la route : Route_B_5 = 1.0


La syntaxe des problèmes linéaire peut être organisée différemment selon les auteurs des programmes, voici l'exemple de la production agrégée de Aaon Stubberfield ou il associe aussi des tableaux pour créer des variables de décision, mais de façon différente que le premier exemple .

D'autant plus qu'un code est toujours l'application d'un modèle mathématique écrit, mais là, cela devient difficile, il faut s'habituer et prendre le temps de lire et de comprendre. On est censé savoir faire le va et vient entre le modèle mathématique et le code, et vice-versa.


In [11]:
from pulp import *
demand = {'A':[5,0,0],'B':[8,7,6]}
costs = {'A':[20,17,18],'B':[15,16,15]}

# On crée le problème
model = LpProblem("Aggregate_Production_Planning",LpMinimize)

# On définit les variables
time = [0, 1, 2]
prod = ['A','B']

# On crée un dictionnaire de toutes les variables de décision de productions possibles et leur temps
#### C'est là que l'on crée des variables de décision en associant les 2 tableaux.***
X = LpVariable.dicts("prod", [(p, t) for p in prod for t in time],lowBound=0, cat="Integer")
# On regarde l'association créee :
print(X)

# On crée la fonction objectif : minimiser les coûts
model += lpSum([costs[p][t] * X[(p, t)] for p in prod for t in time])

# On définit la contrainte que la production >= demande
for p in prod:
    for t in time:
        model += X[(p, t)] >= demand[p][t] 

# Résoudre
model.solve()

# On imprime les variables qui ont leur valeur optimisées
for v in model.variables():
    print(v.name, "=", v.varValue)
    
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Coût total = ", value(model.objective))

{('A', 0): prod_('A',_0), ('A', 1): prod_('A',_1), ('A', 2): prod_('A',_2), ('B', 0): prod_('B',_0), ('B', 1): prod_('B',_1), ('B', 2): prod_('B',_2)}
prod_('A',_0) = 5.0
prod_('A',_1) = 0.0
prod_('A',_2) = 0.0
prod_('B',_0) = 8.0
prod_('B',_1) = 7.0
prod_('B',_2) = 6.0
Coût total =  422.0


# 5. <b>Utiliser une matrice dans les contraintes</b>

Parfois, des matrices sont utiles pour associer par exemple deux modèles de données par un caractère particulier .
Exemple, j'ai 4 développeurs, et 5 langages qu'il pratiquent ou pas, on peut résumer cet état de fait par une matrice aux valeurs binaires.

Voici un exemple de code Python Pulp mettant en jeu une matrice.

Pour plus de détails, on la retrouve dans mon fichier 03, optimisation réseaux, Partie " Sélection d'employés par couverture minimale".
En d'autres termes, cela permet d'employer le minimum de personnes tout en couvrant tous les langages nécessaires au projet.

In [12]:

from pulp import *
 
# les variables de décision sont les employés
employes=["COLLIN","BOB","ALICE","DAVE"]
langages=["PYTHON","RUBY","C++","JAVA","C"]

# Matrice des aptitudes
           # COLLIN , BOB, ALICE, DAVE
aptitudes = [[1,0,0,0], #python
             [1,0,0,0], # Ruby
             [1,1,1,0], # C++
             [0,1,0,1], # Java
             [0,0,1,1] # C
             ]




# On définit votre problème de minimisation
Problem = LpProblem('couverture_minimale_dEmployes',LpMinimize)

# A la place d'écrire en dur les variables de décision, on laisse Pulp les créér automatiquement
# à l'aide du tableau employes.
vars = LpVariable.dicts("EMPLOYE", employes, 0, None, LpBinary)

# On créée la fonction objectif qui est la même que celle écrite en dur
Problem += lpSum(vars[i] for i in employes)

# Pour les contraintes, on utilise énumérate 
# pour faire la liaison entre les indices de type nombre entier(idx) et
# les indices de type STRING ( Les noms des employés)

# On les affiche pour le fun
print (" Lien entre les index ")
for idx, x in enumerate(employes):
    print(idx, x)

# On créée la contrainte en une seule ligne
for j in aptitudes:# Pour chaque ligne de la matrice aptitudes
    Problem += lpSum(j[idx] * vars[i] for idx,i in enumerate(employes)) >= 1 

# La contrainte précédente veut dire : 
# Multiplie la valeur de l'index  de la ligne de la matrice par la variable de décision du même rang
# et ça doit être supérieur ou égal à 1. et cela pour chaque ligne de la matrice.


print("-------------Résultat----------------")
# On résouds le pb, et on peut choisir le solveur entre les parenthèses
Problem.solve()

print('nombre d employés minimisé = ', value(Problem.objective))
 
for i in Problem.variables():
    print('Selection de l employé:',i.name, '=', i.varValue)

 Lien entre les index 
0 COLLIN
1 BOB
2 ALICE
3 DAVE
-------------Résultat----------------
nombre d employés minimisé =  2.0
Selection de l employé: EMPLOYE_ALICE = 0.0
Selection de l employé: EMPLOYE_BOB = 0.0
Selection de l employé: EMPLOYE_COLLIN = 1.0
Selection de l employé: EMPLOYE_DAVE = 1.0


# 6. Itérer avec Python Gekko , le  solveur non  linéaire.

C'est différent d'avec Pulp .

Lien : https://stackoverflow.com/questions/64542594/how-could-constraints-be-dynamically-constructed-in-gekko

Voici un exemple de réduction de coût avec une itération dans la contrainte de temps de travail :

In [13]:
from gekko import GEKKO

# stored as list
my_vars = ['x1','x2']
# stored as dictionaries
Cost = {'x1':100,'x2':125}
Min = {'x1':0,'x2':0}
Max = {'x1':70,'x2':40}
Work = {'x1':50,'x2':50}

LP = GEKKO(remote=False)


va = LP.Array(LP.Var, (len(my_vars)))  # array

# Le truc qui diffère , en fait il crée un dictionnaire à l'aide du tableau créé juste avant :
vd = {}                                # dictionary
for i,xi in enumerate(my_vars):
    vd[xi] = va[i]
    vd[xi].lower = Min[xi]
    vd[xi].upper = Max[xi]


# Fonction coût
LP.Minimize(LP.sum([Cost[xi]*vd[xi] for xi in my_vars])) 


# On voit que l'itération fonctionne !
LP.Equation(LP.sum([Work[xi]*vd[xi] for xi in my_vars])>=200)


LP.solve(disp=False)

# On affiche le résultat
for xi in my_vars:
    print(xi,vd[xi].value[0])
print ('Cost: ' + str(LP.options.OBJFCNVAL))

x1 3.9999999998
x2 1.8225924919e-09
Cost: 400.00000021
